In [ ]:
import pyspark 

In [ ]:
conf = pyspark.SparkConf()
conf.setMaster('spark://spark-master:7077')
spark_context = pyspark.SparkContext(conf=conf) 

In [ ]:
def calculate_weight(helpful_yes, helpful_no):
    def _get_int(st: str) -> int:
        try:
            return int(st.split('.')[0])
        except ValueError:
            return 1
    try:
        return _get_int(helpful_yes) / (_get_int(helpful_yes) + _get_int(helpful_no))
    except ZeroDivisionError:
        return 0

In [ ]:
data = spark_context.textFile('/user-data/combined/reviews.csv')
header_data = data.first()
result_data = data.filter(lambda row: row != header_data) \
    .map(lambda line: line.split(",")) \
    .filter(lambda line: len(line) > 5) \
    .filter(lambda line: line[4].isdigit()) \
    .map(lambda line: (line[1], int(line[4]) * calculate_weight(line[6], line[7]))) \
    .map(lambda line: (line[0], (line[1], 1))) \
    .reduceByKey(lambda val1, val2: (val1[0] + val2[0], val1[1] + val2[1])) \
    .mapValues(lambda x: x[0] / x[1])

    

In [ ]:
print(result_data.count())

In [ ]:
result_data.take(10)